In [129]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg,col,first,when,udf
from pyspark.sql.types import StringType
ss = SparkSession.builder.getOrCreate()

In [130]:
df = ss.read.format("csv").options(header='true', inferSchema='true', delimiter=',').load("df.csv")

#df.show()

In [131]:
dfPivot = df.groupBy("NumEtiqueta").pivot("CoEtiqueta").agg({"idRep": "avg"})
#dfPivot.show()

In [132]:
#Se realiza el pivote del data frame usando como pivote las etiquetas
dfGroup = df.groupBy("idRep","FolioBC","CoCliente","NumEtiqueta").pivot("CoEtiqueta").agg(first("ValorEtiqueta"))
dfGroup.show()

+-----+-------+---------+-----------+---+---+--------+--------+
|idRep|FolioBC|CoCliente|NumEtiqueta|  1|  2|       3|       4|
+-----+-------+---------+-----------+---+---+--------+--------+
|    1|   1234|       17|          1| 01| AA|01031993|1111U211|
|    1|   1234|       17|          2| 02| AB|02041993|1711U211|
|    1|   1234|       17|          3| 03| AC|03051993|1721U211|
|    2|   1235|       34|          1| 01| AD|01031993|1111U211|
|    2|   1235|       34|          2| 02| AE|02041993|1711U211|
|    2|   1235|       34|          3| 03| AF|03051993|1721U211|
+-----+-------+---------+-----------+---+---+--------+--------+



In [135]:
dfFiltrado = dfGroup.filter((col("1") == "01") & (col("2") == "AA"))
#dfFiltrado.show()

In [156]:
def reg1(col1, col2, valores_permitidos):
    if col1 == '-' or col2 == '-':
        return '0'
    if col1 == '01' and col2 in valores_permitidos:
        return '1'
    else:
        return '0'

In [157]:
# Registrar la función como una UDF (User Defined Function)
reg1_udf = udf(lambda col1, col2: reg1(col1, col2, ["AA", "AC", "AE"]), StringType())


In [159]:
## Aplicar el filtro y agregar la nueva columna usando la función reg1
result_df = dfGroup.withColumn("malBuro", reg1_udf(col("1"), col("2")))


result_df.show()


+-----+-------+---------+-----------+---+---+--------+--------+-------+
|idRep|FolioBC|CoCliente|NumEtiqueta|  1|  2|       3|       4|malBuro|
+-----+-------+---------+-----------+---+---+--------+--------+-------+
|    1|   1234|       17|          1| 01| AA|01031993|1111U211|      1|
|    1|   1234|       17|          2| 02| AB|02041993|1711U211|      0|
|    1|   1234|       17|          3| 03| AC|03051993|1721U211|      0|
|    2|   1235|       34|          1| 01| AD|01031993|1111U211|      0|
|    2|   1235|       34|          2| 02| AE|02041993|1711U211|      0|
|    2|   1235|       34|          3| 03| AF|03051993|1721U211|      0|
+-----+-------+---------+-----------+---+---+--------+--------+-------+

